In [1]:
import numpy as np
import random
import os
import glob
import tensorflow as tf
import math
import pickle
#import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw

In [2]:
loadname = '../../class/t0626.p'
class signLClass:
        def __init__(self):
                self.data = [np.zeros(1),np.zeros(1)]
                self.testdata = [np.zeros(1),np.zeros(1)]
                self.labels = []
                self.testlabels = []
                self.first = True
                self.testfirst = True
        def add_image(self, image, label):
                if self.first == False:
                        tmp_image = self.data[1]
                        tmp_label = self.data[0]
                self.data[1] = image
                self.data[0] = label
                #self.data[1] = np.reshape(self.data[1], [-1, 50, 60, 3])
                #self.data[0] = self.data[0].reshape((1,kind))
                if self.first == False:
                        self.data[1] = np.vstack((tmp_image, self.data[1]))
                        self.data[0] = np.vstack((tmp_label, self.data[0]))
                self.first = False
        def add_test_image(self, image, label):
                if self.testfirst == False:
                        tmp_image = self.testdata[1]
                        tmp_label = self.testdata[0]
                self.testdata[1] = image
                self.testdata[0] = label
                #self.data[1] = np.reshape(self.data[1], [-1, 50, 60, 3])
                #self.data[0] = self.data[0].reshape((1,kind))
                if self.testfirst == False:
                        self.testdata[1] = np.vstack((tmp_image, self.testdata[1]))
                        self.testdata[0] = np.vstack((tmp_label, self.testdata[0]))
                self.testfirst = False
        def train(self, n, batch_size):
                return signL.data[1][n:n+batch_size]
        def label(self, n, batch_size):
                return signL.data[0][n:n+batch_size]

def load_class(signL, loadname):
    signL = pickle.load(open(loadname, "rb"))
    return signL
signL = signLClass()
signL = load_class(signL, loadname)
print "Training data class named 'signL'. "

Training data class named 'signL'. 


In [3]:
print signL.data[0].shape

(186209, 54)


In [4]:
# Parameters
learning_rate = 0.001
training_iters = 1600000
batch_size = 16
display_step = 100
SEED = 448

# Network Parameters
n_input = 9000 # MNIST data input (img shape: 50*60*3)
n_classes = len(signL.labels) # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

In [5]:
def weight_variable(shape):
    initial = tf.random_normal(shape, stddev=0.05, dtype=tf.float32)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0, shape=shape,dtype=tf.float32)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_3x3(x):
    return tf.nn.max_pool(x, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')

In [6]:
x = tf.placeholder(tf.float32, [None, 50, 60, 3], name = 'placeholder_x')
#x_28 = tf.reshape(x, shape=[-1, 28, 28, 3])
x_28 = tf.image.resize_images(x, [28,28])

y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32, name = 'keep_prob') #dropout (keep probability)
#x_image = tf.reshape(x, [-1, 28, 28, 1])

W_conv1 = weight_variable([5,5, 3, 192])
b_conv1 = tf.Variable(tf.random_normal([192], stddev=0.01, dtype=tf.float32), name = 'Variable')
output = tf.nn.relu(conv2d(x_28, W_conv1) + b_conv1)
## MLP-1-1##
W_MLP11 = weight_variable([1,1, 192, 160])
b_MLP11 = bias_variable([160])
output = tf.nn.relu(conv2d(output, W_MLP11) + b_MLP11)
## MLP-1-2##
W_MLP12 = weight_variable([1,1, 160, 96])
b_MLP12 = bias_variable([96])
output = tf.nn.relu(conv2d(output, W_MLP12) + b_MLP12)
##Max pooling##
output = max_pool_3x3(output)                     
## dropout ##
output = tf.nn.dropout(output, keep_prob)

## conv2 layer ##
W_conv2 = weight_variable([5,5, 96, 192])
b_conv2 = tf.Variable(tf.random_normal([192], stddev=0.01, dtype=tf.float32), name = 'Variable')
output = tf.nn.relu(conv2d(output, W_conv2) + b_conv2)
## MLP-2-1##
W_MLP21 = weight_variable([1,1, 192, 192])
b_MLP21 = bias_variable([192])
output = tf.nn.relu(conv2d(output, W_MLP21) + b_MLP21)
## MLP-2-2##
W_MLP22 = weight_variable([1,1, 192, 192])
b_MLP22 = bias_variable([192])
output = tf.nn.relu(conv2d(output, W_MLP22) + b_MLP22)
##Max pooling##
output = max_pool_3x3(output)    
## dropout ##
output = tf.nn.dropout(output, keep_prob)

## conv3 layer ##
W_conv3 = weight_variable([3,3, 192, 192])
b_conv3 = tf.Variable(tf.random_normal([192], stddev=0.01, dtype=tf.float32), name = 'Variable')
output = tf.nn.relu(conv2d(output, W_conv3) + b_conv3)
## MLP-2-1##
W_MLP31 = weight_variable([1,1, 192, 192])
b_MLP31 = bias_variable([192])
output = tf.nn.relu(conv2d(output, W_MLP31) + b_MLP31)
## MLP-2-2##
W_MLP32 = weight_variable([1,1, 192, n_classes])
b_MLP32 = bias_variable([n_classes])
output = tf.nn.relu(conv2d(output, W_MLP32) + b_MLP32)
##global average##
output = tf.nn.avg_pool(output, ksize=[1,7,7,1], strides=[1,1,1,1], padding='VALID')
# [n_samples, 1, 1, 10] ->> [n_samples, 1*1*10]

with tf.name_scope("pred"):
    output = tf.reshape(output, [-1, 1*1*n_classes])

In [7]:
#this code demostrate how to schedule step-wise learning rate in a simple way
learning_rate = tf.placeholder(tf.float32)
# the loss function 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output))
# optimizer Adam
train_step = tf.train.AdamOptimizer(learning_rate, 0.9).minimize(cross_entropy)
# prediction
correct_prediction = tf.equal(tf.argmax(output,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
trainset = ['signL','signL2.p']

In [9]:
# Initializing the variables
init = tf.global_variables_initializer()

In [10]:
saver = tf.train.Saver()
sess=tf.Session()
sess.run(init)
step = 1
bloss = 0
br=0
while step * batch_size < training_iters:
    batch_xs = signL.train(step, batch_size)
    batch_ys = signL.label(step, batch_size)
    sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 0.4, learning_rate: 1e-5})
    if step % display_step == 0:
        # Calculate batch accuracy
        acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
        # Calculate batch loss
        loss = sess.run(cross_entropy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
        print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc)
        if(loss>bloss):
            br+=1
        else:
            br = 0
        bloss = loss
        if(br==8):
            break;
    step += 1
print "Optimization Finished!"
# Test model
test_images = signL.testdata[1]
test_labels = signL.testdata[0]
# Calculate accuracy for 256 mnist test images
print "Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_images, y: test_labels, keep_prob: 1.})

Iter 1600, Minibatch Loss= 4.100258, Training Accuracy= 0.00000
Iter 3200, Minibatch Loss= 3.988804, Training Accuracy= 0.00000
Iter 4800, Minibatch Loss= 4.034755, Training Accuracy= 0.00000
Iter 6400, Minibatch Loss= 3.950185, Training Accuracy= 0.06250
Iter 8000, Minibatch Loss= 3.958410, Training Accuracy= 0.00000
Iter 9600, Minibatch Loss= 3.988078, Training Accuracy= 0.00000
Iter 11200, Minibatch Loss= 3.963798, Training Accuracy= 0.06250
Iter 12800, Minibatch Loss= 3.957526, Training Accuracy= 0.06250
Iter 14400, Minibatch Loss= 3.991432, Training Accuracy= 0.00000
Iter 16000, Minibatch Loss= 4.013293, Training Accuracy= 0.00000
Iter 17600, Minibatch Loss= 4.002620, Training Accuracy= 0.00000
Iter 19200, Minibatch Loss= 3.990267, Training Accuracy= 0.00000
Iter 20800, Minibatch Loss= 3.968761, Training Accuracy= 0.00000
Iter 22400, Minibatch Loss= 3.983709, Training Accuracy= 0.00000
Iter 24000, Minibatch Loss= 3.983361, Training Accuracy= 0.06250
Iter 25600, Minibatch Loss= 3.9

In [11]:
saver.save(sess, '../Model/NIN-Model-0626')
with open("../demo/labels.p", "wb") as fp:
    pickle.dump(signL.labels, fp)
with open("../qt/labels.p", "wb") as fp:
    pickle.dump(signL.labels, fp)

In [12]:
with open("../demo/labels.p", "rb") as fp:
    labelname = pickle.load(fp)
print labelname

['name', 'hungry', 'want1', '1', 'd', '7', 's', '2', 'y', 'p', 'me', 'likeb2', 'i', '9', 'q', 'why2', 'nob1', 'sorry1', 'thanku1', 'want2', 'w', 'n', 'c', '3', 'g', 'v', 'time1', 'a', 'come', 'you', 'howd2', 'help_h2', 'unders1', 't', 'nob2', '8', 'u', 'x', '6', 'h', 'e', '5', 'o', '4', 'f', 'non', 'unders2', 'm', 'b', 'likeb1', 'r', 'l', 'howd1', 'k']


In [ ]:
def testing(filename):
    stest_images = get_pic(filename)
    print sess.run(tf.argmax(pred,1), feed_dict={x: stest_images, keep_prob: 1.})

In [ ]:
def testing_all(path):
    for filename in glob.glob(os.path.join(path, '*.png')):
        print filename
        stest_images = get_pic(filename)
        print sess.run(tf.argmax(pred,1), feed_dict={x: stest_images, keep_prob: 1.})

In [ ]:
filename = 'rgbdata/testing/6_491.png'
testing(filename)

In [ ]:
path = 'rgbdata/testing'
testing_all(path)